In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rand

from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21:22].to_numpy()

In [4]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [5]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [6]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.cast(tf.math.subtract(y_pred, 42), dtype=tf.float32)
    b = tf.cast(dist, dtype=tf.float32)

    loss = tf.math.add(a, b)
    return loss

In [7]:
input_layer = tf.keras.layers.Input(shape=(21,))

encoded = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(32, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(16, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(3)(encoded)
 
decoded = tf.keras.layers.Dense(3, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(16, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(128, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(21, activation='sigmoid')(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)

# decoder
encoded_input = tf.keras.layers.Input(shape=(3,))

decoded_layer = autoencoder.layers[6](encoded_input)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoded_layer = autoencoder.layers[8](decoded_layer)
decoded_layer = autoencoder.layers[9](decoded_layer)
decoded_layer = autoencoder.layers[10](decoded_layer)
decoded_layer = autoencoder.layers[11](decoded_layer)

decoder = tf.keras.Model(encoded_input, decoded_layer)

In [8]:
autoencoder.compile(optimizer='adam', loss=my_loss, metrics=['mse'])

In [9]:
result = []
pop_result = []
for i in range(1000):
    X_train = X[rand.randint(0, 394)].reshape(-1, 1).T
    X_pred = autoencoder.predict(X_train, verbose=0)

    loss = my_loss(X_train, X_pred)
    autoencoder.train_on_batch(X_pred, X_train)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[0.34241462]], pop: [[40.92142]]
epoch: 1, loss: [[0.52492213]], pop: [[40.913704]]
epoch: 2, loss: [[0.32591522]], pop: [[40.905994]]
epoch: 3, loss: [[0.5270966]], pop: [[40.89828]]
epoch: 4, loss: [[0.08453405]], pop: [[40.890575]]
epoch: 5, loss: [[0.5317775]], pop: [[40.882816]]
epoch: 6, loss: [[0.2951088]], pop: [[40.875282]]
epoch: 7, loss: [[0.18944192]], pop: [[40.86766]]
epoch: 8, loss: [[0.34184325]], pop: [[40.86003]]
epoch: 9, loss: [[-0.27048522]], pop: [[40.85242]]
epoch: 10, loss: [[-0.07092714]], pop: [[40.84481]]
epoch: 11, loss: [[0.25321913]], pop: [[40.837215]]
epoch: 12, loss: [[0.31123424]], pop: [[40.829605]]
epoch: 13, loss: [[0.42720103]], pop: [[40.822]]
epoch: 14, loss: [[-0.07323563]], pop: [[40.81438]]
epoch: 15, loss: [[-0.29973257]], pop: [[40.806747]]
epoch: 16, loss: [[0.25108433]], pop: [[40.796825]]
epoch: 17, loss: [[0.2749107]], pop: [[40.791515]]
epoch: 18, loss: [[0.20364308]], pop: [[40.783897]]
epoch: 19, loss: [[-0.35145646]]

In [10]:
gen = autoencoder.predict(X)

13/13 [==============================] - 0s 2ms/step


In [11]:
print(dnn_model.predict(gen, verbose=0).max())
print(dnn_model.predict(gen, verbose=0).min())
print(dnn_model.predict(gen, verbose=0).mean())

34.21697
34.213703
34.216816


In [17]:
latest = np.array([X[-1]])
latest_gen = autoencoder.predict(latest)
dist = np.sqrt(np.sum(np.square(latest-latest_gen)))

print(dist)

1/1 [==============================] - 0s 17ms/step
1.5505038769528867


In [18]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [19]:
df = np.append(X_result, pop_result, axis=1)

In [21]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/ae_formula.xlsx')